In [ ]:
# PSSSSS! READ ME! 
# purpose of these code: given a movie list (must be provided in csv format), this program, extracts detail information about 
# the listed movies from two websites using Selenium. The output of the program is a new csv file with the movies details.  

# websites:
    # https://www.imdb.com/
    # https://www.rottentomatoes.com/

# input --> 
    # csv file with the following columns:
        # id_imdb
        # type
        # name
        # release_year
        # release_month
        # genre
    # ⚠️⚠️ important about the input file:
        # the name of the .csv file is entered as an input when the code is executed (NO NEED TO MODIFIED THE CODE)
        # enter only the first part of the name. Example: "list_action_range_1_31" ✅   DO NOT enter the final part "_API.csv" ❌ 
        # the file must be saved in the /data/api folder

# output --> csv file with the following info:
    # IMDB id (this must be provided as en input)
    # IMDB score
    # Rotten Tomatoes score 
    # Director name
    # Screenwriter name
    # Plot
    # Duration (in minutes)
    # Name of the film as it is in IMDB website

# code structure:
    # imports
    # functions definition
        # process input file
        # open website
        # accept cookies
        # search movies
            # retrive movie details 

In [ ]:
# IMPORTS

# import libraries for data processing
# -----------------------------------------------------------------------
import pandas as pd

# Import libraries for web browser automation with Selenium
# -----------------------------------------------------------------------
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # ChromeDriverManager manages the installation of the Chrome driver
from selenium.webdriver.common.keys import Keys # to simulate keyboard events in Selenium
from selenium.webdriver.support.ui import Select # to interact with <select> elements on web pages

# import libraries to pause execution
# -----------------------------------------------------------------------
from time import sleep  # Sleep is used to pause the execution of the program for a number of seconds

# settings
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Sets a Pandas option to display all columns of a DataFrame


In [ ]:
# FUNCTIONS DEFINITION

# process input file
def process_input (file_name_root):
    # open csv file with pandas
    df = pd.read_csv(f"data/api/{file_name_root}_API.csv", index_col = 0)

    # transform into a list of tuples
    list_of_movies_from_api = list(df.itertuples(index=True, name=None))

    return list_of_movies_from_api


# open url function
def open_url(url, driver):
    # maximize window
    driver.maximize_window()
    print("\nMaximizing window 🪟...")

    # navigate to website
    driver.get (url)
    print(f"\nNavigating to ⛵ {url}...")

    # pause to let the website load
    nap_time = 3
    sleep(nap_time)
    print(f"\nTaking a quick nap 😴... Only {nap_time} seconds though!")


# accept cookies function
def accept_cookies(locator):

    try:
        driver.find_element(locator[0], locator[1]).click()
        print("\nCookies accepted 🍪...")
    except:
        print("\nCookies not found ❌...")


# search movies function
def search_movies_imdb (movies_list, search_bar_locator):

    all_movies_data = [] # create a list to store the info to be collected of each movie
    
    for i in range (len(movies_list)):
        movie_id = movies_list [i][0]
        movie_name = movies_list [i][2]
        print (f"\n<< Iteration {i} >>\nSearching for movie 🔍 '{movie_name}', id '{movie_id}'...")
        #search_bar_locator = ["css selector", "#suggestion-search"]
        try:
            driver.find_element(search_bar_locator[0],search_bar_locator[1]).send_keys(movie_id, Keys.ENTER)
            print(f"'{movie_name}' found! ✅")
        except:
            try: 
                driver.back() # this try fixis the cases where a previous individual movie page threw a 404 Error. In these cases the search bar is not reachable. Navigating back solves the issue 
                driver.find_element(search_bar_locator[0],search_bar_locator[1]).send_keys(movie_id, Keys.ENTER)
                print(f"'{movie_name}' found! ✅")
            except:
                print (f"Couldn't find movie '{movie_name}', id '{movie_id}' ❌")
                continue
        
        # call function "retrive_movie_data" to get the data related to the movie
        movie_data = retrive_movie_data(movie_id)
        
        # append movie info into final list 
        all_movies_data.append(movie_data)
        
    print("Seach completed ✅")
    return all_movies_data


# retrive movies details function
def retrive_movie_data(movie_id):
        
        movie_data = [] # create an empty list to store info on each iteration. It's a temporary list that will be overwritten on each iteration

        print ("\nExtracting data👽...")

        imdb_score_locator = ["xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]']
        director_locator = ["css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(1) > div > ul > li > a"]
        screenwriters_locator = ["css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul > li:nth-child(1) > a"]
        duration_locator = ["css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > ul > li:nth-child(2)"]
        imdb_name_locator = ["css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-1f50b7c-0.PUxFE > h1 > span"]

        # imdb score
        try:
            imdb_score = driver.find_element(imdb_score_locator[0], imdb_score_locator[1]).text
            print (" - IMDB rating ⭐ found✅")
        except:
            imdb_score = "none"
            print (" - IMDB rating not found❌")
        
        movie_data.append(imdb_score)

        # director
        try:
            director = driver.find_element(director_locator[0], director_locator[1]).text
            print (" - Director 🎬 found✅")
        except:
            director = "none"
            print (" - Director not found ❌")

        movie_data.append(director)

        # screenwriters
        try:
            screenwriters = driver.find_element(screenwriters_locator[0], screenwriters_locator[1]).text
            print (" - Screenwriters ✍️ found✅")
        except:
            screenwriters = "none"
            print (" - Screenwriters not found ❌")

        movie_data.append(screenwriters)

        # duration
        try:
            duration = driver.find_element(duration_locator[0], duration_locator[1]).text
            print (" - Duration ⏱️ found✅")
        except:
            duration = "none"
            print (" - Duration not found ❌")
        
        if duration != "none":
        #transform duration format from 1h 44min into INT (minutes)
        # get the number of hours
            if "h" in duration:
                h_position = duration.index("h")
                hours = int(duration[:h_position])
            else:
                hours = 0

            # get the number of minutes
            if "min" in duration:
                min_position = duration.index("min")
                minutes = int(duration [min_position -2 : min_position])
            else:
                minutes = 0
            
            duration_in_minutes = hours * 60 + minutes

            print (f"  -> Movie duration transformed from {duration} into {duration_in_minutes}, now we have duration in {type(duration_in_minutes)}! 🎉")
        
        else:
            duration_in_minutes = duration
        
        movie_data.append(duration_in_minutes)

        # imdb_name
        try:
            imdb_name = driver.find_element(imdb_name_locator[0], imdb_name_locator[1]).text
            print (" - IMDB name 🅰️ found✅")
        except:
            imdb_name = "none"
            print (" - IMDB name not found❌")

        movie_data.append (imdb_name)

        movie_data.append (movie_id)

        #movies_full_info.append (movie_info)

        return movie_data



In [ ]:
# CODE EXECUTION BELOW ⬇️⬇️⬇️ 

# Summary:
    # 1) process input file
    # 2) open browser
    # 3) open url (imdb website)
    # 4) accept cookies 
    # 5) search movies and retrieve movies details
    # 6) open url (rotten tomatoes)
    # 7) accept cookies
    # 8) search movies and retrieve additional movies details
    # 9) process output

In [ ]:
# 1) process input file

#ask user for the file name
file_name_root = input("🗃️ Please enter the csv file name. \n⚠️IMPORTANT⚠️ Enter the name without the '_API.csv' part.\n EXAMPLE: 'list_action_range_1_31'")

# call process_input function
list_of_movies_from_api = process_input (file_name_root)

In [ ]:
# 2) open browser

driver = webdriver.Chrome()

print("\nOpening Chrome 🧭...")

In [ ]:
# 3) open url (imdb)

url = "https://www.imdb.com/"

open_url (url, driver)

In [ ]:
# 4) accept_cookies
 
imdb_cookies_locator = ["css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"]

accept_cookies(imdb_cookies_locator)

In [ ]:
# 5) search movies on imdb website and retrieve movies data

search_bar_locator = ["css selector", "#suggestion-search"]

all_movies_data = search_movies_imdb(list_of_movies_from_api, search_bar_locator)

# print list for testing
print(all_movies_data)

In [ ]:
# 6) open url (rotten  tomatoes)

url = "https://www.rottentomatoes.com/"

open_url (url, driver)

In [ ]:
# 7) accept cookies

rt_cookies_locator = ["css selector", "#onetrust-accept-btn-handler"]

accept_cookies(rt_cookies_locator)

In [ ]:
# 8) search movies and retrieve additional movies details
# PENDING or Improvement for next phase: create function to search in Rotten Tomatoes page
 
for i in range (len(list_of_movies_from_api)):

    # find the movie
    movie_name = list_of_movies_from_api [i][2]
    print (f"\n<< Iteration {i} >>\nSearching for movie 🔍 '{movie_name}'...")

    # add inputs on search bar
    search_bar_locator_1 = ["css selector", "#header-main > search-results-nav > search-results-controls > input[type=text]"]
    try:
        driver.find_element(search_bar_locator_1[0], search_bar_locator_1[1]).send_keys(movie_name, Keys.ENTER)
    except:
        try: # this try fixis the cases where a previous individual movie page threw a 404 Error. In these cases the search bar is not reachable. Navigating back solves the issue
                driver.back()
                driver.find_element(search_bar_locator_1[0], search_bar_locator_1[1]).send_keys(movie_name, Keys.ENTER)
        except:
            print ("Error related to finding or sending inputs thorugh the search bar⚠️")
            continue
    
    # navigate the results
    # click on "Movies" to filter results to show Movies only
    movies_filter_locator = ["css selector", "#search-results > nav > ul > li:nth-child(2) > span"]
    driver.find_element(movies_filter_locator[0], movies_filter_locator[1]).click()
    print ("\nFiltering results by 'movie'...\n")

    # iterate over the first five search results 
    movie_found = False
    for x in range (1,6):
        # get movie name as in Rotten Tomatoes website and compare the name with the movie name from lista_general
        try: 
            result_name_locator = ["css selector", f"#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child({x}) > a:nth-child(2)"]
            rt_name = driver.find_element(result_name_locator[0], result_name_locator[1]).text.lower()
        except:
            print ("Error accesing the movie name in the result page ⚠️")
            break
        print (f">> {x}/5 Comparing result '{rt_name}'")                                                   
        if rt_name == movie_name.lower():
            print(f"'{movie_name}' found! ✅")
            # if result name matches movie name, click on result name to access movie individual page
            try: 
                driver.find_element(result_name_locator[0], result_name_locator[1]).click()
            except:
                print ("Error when clicking movie name ⚠️")
                break
            try:
                rt_score_locator = ["css selector","#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text"]
                rt_score = driver.find_element (rt_score_locator[0], rt_score_locator[1]).text
                # transform tomatometer into INT
                rt_score_as_int = int(rt_score.replace("%",""))
                print (" - Score 🍅 found✅")
            except:
                rt_score_as_int = "none"
                print (" - Score not found ❌")
            try:
                plot_locator = ["css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > drawer-more > rt-text"]
                rt_plot = driver.find_element (plot_locator[0], plot_locator[1]).text
                print (" - Plot 📰 found✅")
            except:
                rt_plot = "none"
                print (" - Plot not found ❌")
            movie_found = True
            break

    if not movie_found: 
        print ("movie not found")
        rt_score_as_int = "none"
        rt_plot = "none"

    all_movies_data[i].insert(1,rt_score_as_int)
    all_movies_data[i].insert(4,rt_plot)

In [ ]:
# 9) process output

# transformation to tuples
for i in range (len(all_movies_data)):
    all_movies_data[i] = tuple (all_movies_data[i])

# print for testing 
print (all_movies_data)

# transform into a DataFrame
columns =  ['score_imdb', 'score_rt', 'director_imdb', 'screenriters_imdb', 'plot_rt', 'duration_imdb', 'title_imdb', 'id_imdb']
df_movies_details = pd.DataFrame(all_movies_data, columns = columns)
df_movies_details.set_index('id_imdb', inplace=True)
df_movies_details

# save into a .csv file
df_movies_details.to_csv(f"data/selenium_movies/{file_name_root}_sel_movies.csv")
